In [1]:
import sys
import os

path = os.getcwd()

from collections import Counter

import matplotlib
import matplotlib.pyplot as plt

import numpy as np

import pandas as pd

import scipy.stats

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.dummy import DummyClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, PowerTransformer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import brier_score_loss, make_scorer
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.pipeline import Pipeline

matplotlib.rcParams.update({'xtick.labelsize': 16})
matplotlib.rcParams.update({'ytick.labelsize': 16})

In [2]:
# Read in the csv file
df = pd.read_csv(path + '/' + 'haberman.data.csv')

In [6]:
data = df.values

# split into input and output elements
X, y = data[:, :-1], data[:, -1]

# label encode the target variable to have the classes 0 and 1
y = LabelEncoder().fit_transform(y)


In [7]:
# calculate brier skill score (BSS)
def brier_skill_score(y_true, y_prob):
    # calculate reference brier score
    ref_probs = [per_pos for _ in range(len(y_true))]
    bs_ref = brier_score_loss(y_true, ref_probs)
    # calculate model brier score
    bs_model = brier_score_loss(y_true, y_prob)
    # calculate skill score
    print(1.0 - (bs_model / bs_ref))
    return 1.0 - (bs_model / bs_ref)


In [8]:
# evaluate a model
def evaluate_model(X, y, model):
    # define evaluation procedure
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # define the model evaluation the metric
    metric = make_scorer(brier_skill_score, needs_proba=True)
    # evaluate model
    scores = cross_val_score(model, X, y, scoring=metric, cv=cv, n_jobs=-1)
    return scores


In [14]:
# fit the model
model = Pipeline(steps=[('t1', MinMaxScaler()), ('t2', PowerTransformer()),('m',LogisticRegression(solver='lbfgs'))])
model.fit(X, y)

print('Model fit complete.')


Model fit complete.


In [25]:
age = float(input('What was the age of the patient at the time of the operation?: '))
year = float(input('In what year was the surgery performed?: ')) - 1900.0
nodes = float(input('How many positive axillary nodes were detected in the patient?: '))
# some new cases
data = [[age,year,nodes]]
for row in data:
    # make prediction
    yhat = model.predict_proba([row])
    # get percentage of survival
    p_survive = yhat[0, 0] * 100
    # summarize
    print('\n')
    print('\n')
    print('A {0:.0f}-year-old patient undergoing the breast cancer surgery in {1:.0f}'.format(age, year + 1900.0))
    print('with {0:.0f} positive axillary nodes detected has a {1:.2f}% chance of survival.'.format(nodes, p_survive))
    

What was the age of the patient at the time of the operation?: 26
In what year was the surgery performed?: 1994
How many positive axillary nodes were detected in the patient?: 5




A 26-year-old patient undergoing the breast cancer surgery in 1994
with 5 positive axillary nodes detected has a 80.96% chance of survival.
